In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

In [2]:
iris = load_iris(as_frame=True)

In [3]:
df = iris.frame
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
df["target_name"] = df["target"].map(dict(enumerate(iris.target_names)))

In [5]:
df_filtered = df[df["target_name"].isin(["versicolor", "virginica"])]

In [6]:
df_filtered.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name
50,7.0,3.2,4.7,1.4,1,versicolor
51,6.4,3.2,4.5,1.5,1,versicolor
52,6.9,3.1,4.9,1.5,1,versicolor
53,5.5,2.3,4.0,1.3,1,versicolor
54,6.5,2.8,4.6,1.5,1,versicolor
55,5.7,2.8,4.5,1.3,1,versicolor
56,6.3,3.3,4.7,1.6,1,versicolor
57,4.9,2.4,3.3,1.0,1,versicolor
58,6.6,2.9,4.6,1.3,1,versicolor
59,5.2,2.7,3.9,1.4,1,versicolor


In [7]:
class MyLogisticRegression:
    def __init__(self, lr=0.1, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
        self.w = None
        self.b = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        for _ in range(self.n_iter):
            linear = np.dot(X, self.w) + self.b
            y_pred = self.sigmoid(linear)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        linear = np.dot(X, self.w) + self.b
        y_pred = self.sigmoid(linear)
        return (y_pred >= 0.5).astype(int)

In [16]:
from sklearn.model_selection import train_test_split

X = df_filtered.drop(columns=["target", "target_name"]).values
y = (df_filtered["target_name"] == "virginica").astype(int).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_gd = MyLogisticRegression(lr=0.1, n_iter=2000)
model_gd.fit(X_train, y_train)

y_pred = model_gd.predict(X_test)
accuracy = np.mean(y_pred == y_test)

print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.850


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import RMSprop

X = df_filtered.drop(columns=["target", "target_name"]).values
y = (df_filtered["target_name"] == "virginica").astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

model_rms = Sequential([
    Input(shape=(X_train.shape[1],)),  
    Dense(1, activation='sigmoid')])

model_rms.compile(
    optimizer=RMSprop(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy'])

model_rms.fit(X_train, y_train, epochs=200, verbose=0)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy (RMSProp via Keras): {accuracy:.3f}")

Accuracy (RMSProp via Keras): 0.900


In [21]:
from tensorflow.keras.optimizers import Nadam

X = df_filtered.drop(columns=["target", "target_name"]).values
y = (df_filtered["target_name"] == "virginica").astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

model_nadam = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')])

model_nadam.compile(
    optimizer=Nadam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy'])

model_nadam.fit(X_train, y_train, epochs=200, verbose=0)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy (Nadam via Keras): {accuracy:.3f}")

Accuracy (Nadam via Keras): 0.900


In [14]:
import time

results = {}

In [30]:
from sklearn.metrics import accuracy_score
import time

results = {}

# Gradient Descent
start_time = time.time()
y_pred_gd = model_gd.predict(X_test)
acc_gd = accuracy_score(y_test, y_pred_gd)
results['Gradient Descent'] = (acc_gd, time.time() - start_time)

# RMSProp
start_time = time.time()
loss, acc = model_rms.evaluate(X_test, y_test, verbose=0)
results['RMSProp'] = (acc, time.time() - start_time)

# Nadam
start_time = time.time()
loss, acc = model_nadam.evaluate(X_test, y_test, verbose=0)
results['Nadam'] = (acc, time.time() - start_time)

print("| Method | Accuracy | Working time (sec) |")
print("|-------|---------|------------------|")
for method, (acc, t) in results.items():
    print(f"| {method} | {acc:.3f} | {t:.2f} |")

| Method | Accuracy | Working time (sec) |
|-------|---------|------------------|
| Gradient Descent | 0.850 | 0.00 |
| RMSProp | 0.800 | 0.07 |
| Nadam | 0.800 | 0.03 |
